## Exercise 7 - Final

This final exercise combines different parts from the previous exercises.

### Section 1

1 Search Digitraffic's REST interface service (in JSON format) for active trains with the terminal address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [86]:
import requests

station = 'HKI' # Helsinki 

arrived_trains = 10
departed_trains = 10
include_nonstopping = False

params = {
    'arrived_trains': arrived_trains,
    'departed_trains': departed_trains,
    'include_nonstopping': include_nonstopping
}


url = f"https://rata.digitraffic.fi/api/v1/live-trains/station/{station}/"

req = requests.get(url, params)
req.status_code

200

2 Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [87]:
import pandas as pd
df = pd.json_normalize(req.json())

df.head()

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,12,2024-02-18,10,vr,IC,Long-distance,,False,False,287671073832,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'JNS', 'stationUICCode':..."
1,273,2024-02-18,10,vr,IC,Long-distance,,True,False,287671177208,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,972,2024-02-18,10,vr,IC,Long-distance,,False,False,287671038059,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'KUT', 'stationUICCode':..."
3,8502,2024-02-18,10,vr,HL,Commuter,L,False,False,287671162644,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'KKN', 'stationUICCode':..."
4,8525,2024-02-18,10,vr,HL,Commuter,L,True,False,287671180487,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


3 Delete all other columns except `timeTableRows` and `trainNumber`.

In [88]:
df = df.loc[:, ['timeTableRows', 'trainNumber']]
df

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'JNS', 'stationUICCode':...",12
1,"[{'stationShortCode': 'HKI', 'stationUICCode':...",273
2,"[{'stationShortCode': 'KUT', 'stationUICCode':...",972
3,"[{'stationShortCode': 'KKN', 'stationUICCode':...",8502
4,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8525
5,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8529
6,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8734
7,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8735
8,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8736
9,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8738


4 After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [89]:
df.iloc[0,0]

[{'stationShortCode': 'JNS',
  'stationUICCode': 460,
  'countryCode': 'FI',
  'type': 'DEPARTURE',
  'trainStopping': True,
  'commercialStop': True,
  'commercialTrack': '1',
  'cancelled': False,
  'scheduledTime': '2024-02-18T16:11:00.000Z',
  'liveEstimateTime': '2024-02-18T16:32:40.000Z',
  'estimateSource': 'LIIKE_USER',
  'actualTime': '2024-02-18T16:34:52.000Z',
  'differenceInMinutes': 24,
  'causes': [{'categoryCode': 'L',
    'detailedCategoryCode': 'L6',
    'thirdCategoryCode': 'L601',
    'categoryCodeId': 352389194,
    'detailedCategoryCodeId': 118817032,
    'thirdCategoryCodeId': 17241345}],
  'trainReady': {'source': 'KUPLA',
   'accepted': True,
   'timestamp': '2024-02-18T16:32:34.000Z'}},
 {'stationShortCode': 'PLT',
  'stationUICCode': 1070,
  'countryCode': 'FI',
  'type': 'ARRIVAL',
  'trainStopping': False,
  'commercialTrack': '',
  'cancelled': False,
  'scheduledTime': '2024-02-18T16:11:42.000Z',
  'liveEstimateTime': '2024-02-18T16:35:34.000Z',
  'estimat

In [90]:
nested_df = pd.json_normalize(df.iloc[0,0])
nested_df

# new_df_columns = nested_df.columns
# new_df_columns

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,differenceInMinutes,causes,trainReady.source,trainReady.accepted,trainReady.timestamp
0,JNS,460,FI,DEPARTURE,True,True,1,False,2024-02-18T16:11:00.000Z,2024-02-18T16:32:40.000Z,LIIKE_USER,2024-02-18T16:34:52.000Z,24,"[{'categoryCode': 'L', 'detailedCategoryCode':...",KUPLA,True,2024-02-18T16:32:34.000Z
1,PLT,1070,FI,ARRIVAL,False,NaN,,False,2024-02-18T16:11:42.000Z,2024-02-18T16:35:34.000Z,LIIKE_AUTOMATIC,NaN,24,[],NaN,NaN,NaN
2,PLT,1070,FI,DEPARTURE,False,NaN,,False,2024-02-18T16:11:42.000Z,2024-02-18T16:35:34.000Z,LIIKE_AUTOMATIC,NaN,24,[],NaN,NaN,NaN
3,SUL,1071,FI,ARRIVAL,False,NaN,,False,2024-02-18T16:12:32.000Z,2024-02-18T16:36:23.000Z,LIIKE_AUTOMATIC,NaN,24,[],NaN,NaN,NaN
4,SUL,1071,FI,DEPARTURE,False,NaN,,False,2024-02-18T16:12:32.000Z,2024-02-18T16:36:23.000Z,LIIKE_AUTOMATIC,NaN,24,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,KÄP,977,FI,ARRIVAL,False,NaN,1,False,2024-02-18T20:37:00.000Z,NaN,NaN,2024-02-18T20:56:05.000Z,19,[],NaN,NaN,NaN
132,KÄP,977,FI,DEPARTURE,False,NaN,1,False,2024-02-18T20:37:00.000Z,NaN,NaN,2024-02-18T20:56:05.000Z,19,[],NaN,NaN,NaN
133,PSL,10,FI,ARRIVAL,True,True,6,False,2024-02-18T20:39:00.000Z,NaN,NaN,2024-02-18T20:58:43.000Z,20,[],NaN,NaN,NaN
134,PSL,10,FI,DEPARTURE,True,True,6,False,2024-02-18T20:40:00.000Z,NaN,NaN,2024-02-18T21:00:13.000Z,20,[],NaN,NaN,NaN


In [91]:
import json

# convert nested JSON into dataframe columns
# add column trainNumber

5 Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [92]:
df = df.drop(['causes', 'differenceInMinutes', 'commercialStop'], axis=1, inplace=True)


KeyError: "['causes', 'differenceInMinutes', 'commercialStop'] not found in axis"

6 Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [ ]:
#

### Section 2

7 Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in step 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [93]:
import re
tr_pattern = r"<tr[^>]*>(.*?)<\/tr>" # table rows, ignore attributes
th_pattern = r"<th>(.*?)<\/th>" # dataframe columns
td_pattern = r"<td>(.*?)<\/td>" # dataframe rows

with open("./data.html") as file:
    html_data = file.read()

columns = re.findall(th_pattern, html_data)

# rows = 
# table_data = []

# table_df = pd.DataFrame

# columns
html_data

'<!DOCTYPE html>\n\n<html>\n<head>\n    <meta charset="utf-8">\n    <style>\n        \n        table, th, td {\n            border: 1px solid black;\n            border-collapse: collapse;\n            }\n\n        table{\n            width: 75%;\n            margin-left: auto;\n            margin-right: auto;\n            text-align: center;\n        }\n\n        td{\n            padding: 5px;\n        }\n\n        #actualTime{\n            background-color: green\n        }\n    </style>\n\n</head>\n\n\n<header>\n\n</header>\n\n\n<body>\n\n    <table>\n        <tr id="actualTime">\n\n                <th>actualTime</th>\n                <th>cancelled</th>\n                <th>commercialTrack</th>\n                <th>countryCode</th>\n                <th>scheduledTime</th>\n                <th>stationShortCode</th>\n                <th>stationUICCode</th>\n                <th>trainStopping</th>\n                <th>type</th>\n                <th>trainNumber</th>\t\t\t\t\t\t\t\t\t\n   

### Section 3

8 Make a SQLite database and insert the dataframe obtained as a result of step 2 to the database.

In [ ]:
import sqlite3

# Create a database connection
db_connection = sqlite3.connect('final.db')

# Save dataframe to database
df.to_sql(name='live_trains', con=db_connection, if_exists='fail', index=False)

# Close the connect
db_connection.close()